# Comparative experiments over GraphRNN, GRAN and GraphOpt
Ousmane TOUAT

## Purpose
Investigating on those deep graph generators methods over small-words graph, in order to design a capable RL-based generator

## Setup
### Libraries import

In [1]:
#Include local libraries
import sys,os
from synthethicDataset.generate_graphs import *
from baselineModels.GraphRNN.train import *
from baselineModels.GraphRNN.model import *
from baselineModels.GraphRNN.data import *
from experiment_config import *
from time import strftime,gmtime
import scipy.misc
from tensorboard_logger import configure, log_value
from random import shuffle
import random
import pickle

## Testing GraphRNN

In [2]:
# All necessary arguments are defined in args.py
args = Args()
os.environ['CUDA_VISIBLE_DEVICES'] = str(args.device)
print('CUDA', args.device)
print('File name prefix', args.fname)
# check if necessary directories exist
if not os.path.isdir(args.model_save_path):
    os.makedirs(args.model_save_path)
if not os.path.isdir(args.graph_save_path):
    os.makedirs(args.graph_save_path)
if not os.path.isdir(args.figure_save_path):
    os.makedirs(args.figure_save_path)
if not os.path.isdir(args.timing_save_path):
    os.makedirs(args.timing_save_path)
if not os.path.isdir(args.figure_prediction_save_path):
    os.makedirs(args.figure_prediction_save_path)
if not os.path.isdir(args.nll_save_path):
    os.makedirs(args.nll_save_path)

time = strftime("%Y-%m-%d %H:%M:%S", gmtime())
# logging.basicConfig(filename='logs/train' + time + '.log', level=logging.DEBUG)
if args.clean_tensorboard:
    if os.path.isdir(args.dir_input+"/tensorboard"):
        shutil.rmtree("tensorboard")
configure(args.dir_input+"/tensorboard/run" + time, flush_secs=5)

graphs = generate(args)

# split datasets
random.seed(123)
shuffle(graphs)
graphs_len = len(graphs)
graphs_test = graphs[int(0.8 * graphs_len):]
graphs_train = graphs[0:int(0.8 * graphs_len)]
graphs_validate = graphs[0:int(0.2 * graphs_len)]

graph_validate_len = 0
for graph in graphs_validate:
    graph_validate_len += graph.number_of_nodes()
graph_validate_len /= len(graphs_validate)
print('graph_validate_len', graph_validate_len)

graph_test_len = 0
for graph in graphs_test:
    graph_test_len += graph.number_of_nodes()
graph_test_len /= len(graphs_test)
print('graph_test_len', graph_test_len)

args.max_num_node = max([graphs[i].number_of_nodes() for i in range(len(graphs))])
max_num_edge = max([graphs[i].number_of_edges() for i in range(len(graphs))])
min_num_edge = min([graphs[i].number_of_edges() for i in range(len(graphs))])

# args.max_num_node = 2000
# show graphs statistics
print('total graph num: {}, training set: {}'.format(len(graphs), len(graphs_train)))
print('max number node: {}'.format(args.max_num_node))
print('max/min number edge: {}; {}'.format(max_num_edge, min_num_edge))
print('max previous node: {}'.format(args.max_prev_node))

# save ground truth graphs
## To get train and test set, after loading you need to manually slice
save_graph_list(graphs, args.graph_save_path + args.fname_train + '0.dat')
save_graph_list(graphs, args.graph_save_path + args.fname_test + '0.dat')
print('train and test graphs saved at: ', args.graph_save_path + args.fname_test + '0.dat')

### dataset initialization
dataset = Graph_to_sequence(graphs_train, max_prev_node=args.max_prev_node,
                                         max_num_node=args.max_num_node)
sample_strategy = torch.utils.data.sampler.WeightedRandomSampler([1.0 / len(dataset) for i in range(len(dataset))],
                                                                 num_samples=args.batch_size * args.batch_ratio,
                                                                 replacement=True)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, num_workers=args.num_workers,
                                             sampler=sample_strategy)

### model initialization
node_level_rnn = RNN(input_size=args.max_prev_node, embedding_size=args.embedding_size_rnn,
                        hidden_size=args.hidden_size_rnn, num_layers=args.num_layers, has_input=True,
                        has_output=True, output_size=args.hidden_size_rnn_output).cuda()
edge_level_rnn = RNN(input_size=1, embedding_size=args.embedding_size_rnn_output,
                           hidden_size=args.hidden_size_rnn_output, num_layers=args.num_layers, has_input=True,
                           has_output=True, output_size=1).cuda()

### start training
train(args, dataset_loader, node_level_rnn, edge_level_rnn)

CUDA 0
File name prefix GraphRNN_erdos_4_128_
graph_validate_len 25.0
graph_test_len 25.0
total graph num: 200, training set: 160
max number node: 25
max/min number edge: 45; 15
max previous node: 20
train and test graphs saved at:  ./baselineModels/GraphRNN/graphs/GraphRNN_erdos_4_128_test_0.dat


/home/ousmanetouat/Documents/Projets Python/graphRL/baselineModels/GraphRNN/model.py:44: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(param,gain=nn.init.calculate_gain('sigmoid'))
/home/ousmanetouat/Documents/Projets Python/graphRL/baselineModels/GraphRNN/model.py:42: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.25)
/home/ousmanetouat/Documents/Projets Python/graphRL/baselineModels/GraphRNN/model.py:47: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  m.weight.data = init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))
/home/ousmanetouat/anaconda3/envs/graphRL/lib/python3.7/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 50/100, train loss: 0.087802, graph type: erdos, num_layer: 4, hidden: 128
test done, graphs saved


KeyboardInterrupt: 